In [1]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [3]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [4]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [5]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [6]:
input_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="family", properties=jdbc_properties))


In [7]:
input_df.show()

+------+-----+---+
|person| type|age|
+------+-----+---+
|    A1|Adult| 54|
|    A2|Adult| 53|
|    A3|Adult| 52|
|    A4|Adult| 58|
|    A5|Adult| 54|
|    C1|Child| 20|
|    C2|Child| 19|
|    C3|Child| 22|
|    C4|Child| 15|
+------+-----+---+



In [12]:
window = Window.orderBy("person")
adult_df = input_df.where("type = 'Adult'").withColumn("row_num", row_number().over(window))

In [13]:
child_df = input_df.where("type = 'Child'").withColumn("row_num", row_number().over(window))

In [21]:
adult_df.alias("t1").join(child_df.alias("t2"), col("t1.row_num") == col("t2.row_num") , 'left').orderBy("t1.person")\
.select("t1.person", "t2.person").show()

+------+------+
|person|person|
+------+------+
|    A1|    C1|
|    A2|    C2|
|    A3|    C3|
|    A4|    C4|
|    A5|  NULL|
+------+------+

